In [1]:
# Import all neccesary python libraries for analysis.
import pymc as pm
import pandas as pd
import pyreadr
import nutpie
import arviz as az

In [2]:
# Import rda file into OrderedDict.
keys = pyreadr.read_r("data/ThirdWave.rda")

# Extract from OrderedDict the pandas dataframe.
df = keys["ThirdWave"]

# Extract the data for analysi
seTE = df["seTE"]
# Convert to integer to make the model code work below.


df['Author'] = pd.Categorical(df['Author'])
author_id = df['Author'].cat.codes
TE = df["TE"]

In [9]:
# Generate pymc Bayesian Hierachial model
with pm.Model() as model:
    
    intercept = pm.Normal("intercept", mu = 0, sigma = 1)
    sigma_a = pm.HalfCauchy("sigma_a", beta=.5)
    # Random intercepts
    alpha = pm.Normal('alpha', mu=0, sigma = sigma_a, shape=len(df["Author"].unique()))
    y = pm.Normal("y", mu = intercept + alpha[author_id], sigma = seTE, observed=TE)
   

In [10]:
# Compile nutpie pymc model.
compiled_model = nutpie.compile_pymc_model(model)
trace = nutpie.sample(compiled_model)

Progress,Draws,Divergences,Step Size,Gradients/Draw
,1300,0,0.62,7
,1300,0,0.57,7
,1300,0,0.58,7
,1300,0,0.62,7
,1300,0,0.57,11
,1300,0,0.61,15


In [11]:
# Get the results of the model MCMC samples.
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.070,0.196,-0.301,0.443,0.002,0.002,7596.0,4183.0,1.0
alpha[1],-0.144,0.174,-0.478,0.175,0.003,0.002,4594.0,4372.0,1.0
alpha[2],0.491,0.285,-0.012,1.025,0.006,0.004,2388.0,3380.0,1.0
alpha[3],-0.117,0.148,-0.399,0.152,0.002,0.002,3951.0,3352.0,1.0
alpha[4],0.036,0.172,-0.279,0.371,0.002,0.002,6241.0,4250.0,1.0
alpha[5],0.090,0.184,-0.273,0.434,0.002,0.002,7108.0,4586.0,1.0
alpha[6],-0.030,0.181,-0.366,0.318,0.002,0.003,6567.0,3757.0,1.0
alpha[7],-0.206,0.170,-0.531,0.105,0.003,0.002,3636.0,3799.0,1.0
alpha[8],0.289,0.242,-0.140,0.751,0.004,0.003,4377.0,4196.0,1.0
alpha[9],-0.309,0.191,-0.665,0.041,0.004,0.003,2874.0,2751.0,1.0


# References
Harrer, M., Cuijpers, P., Furukawa, T.A., & Ebert, D.D. (2021). Doing Meta-Analysis with R: A Hands-On Guide. Boca Raton, FL and London: Chapmann & Hall/CRC Press. ISBN 978-0-367-61007-4.

Röver, Christian. 2017. “Bayesian Random-Effects Meta-Analysis Using the ‘Bayesmeta‘ r Package.” ArXiv Preprint 1711.08683.